<a href="https://colab.research.google.com/github/mublify/ML/blob/main/Spam_Text_Message_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install text-preprocessing

In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer  # Use tensorflow for Tokenizer
nltk.download('punkt')

# Data Cleaning and Preprocessing (ideally in a separate function)
import numpy as np
import collections
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Model Building (ideally in a separate function)
from keras.preprocessing.sequence import pad_sequences
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df=pd.read_csv('stmc.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df.isnull().sum()

,0
Category,0
Message,0


In [ ]:
df=df.drop_duplicates(keep='first')
df


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
# prompt: plotly to piechart distribution of df'Category'. Ham in green and spam in red. Show labels

import plotly.express as px

# Count the occurrences of each category
category_counts = df['Category'].value_counts()

# Create the pie chart using Plotly Express
fig = px.pie(
    category_counts,
    values=category_counts.values,
    names=category_counts.index,
    color=category_counts.index,
    title='Distribution of SMS Categories',
)

# Show the labels
fig.update_traces(textinfo='label+percent', textposition='inside')

fig.show()


In [ ]:

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Target'] = encoder.fit_transform(df['Category'])

In [ ]:
df['num_chars'] = df['Message'].apply(len)
df['num_words'] = df['Message'].apply(lambda x: len(x.split()))
df['num_sentences'] = df['Message'].apply(lambda x: len(re.split(r'[.!?]+', x)))
df

,Category,Message,Target,num_chars,num_words,num_sentences
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,20,4
1,ham,Ok lar... Joking wif u oni...,0,29,6,3
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,28,2
3,ham,U dun say so early hor... U c already then say...,0,49,11,3
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,13,1
...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,160,30,6
5568,ham,Will ü b going to esplanade fr home?,0,36,8,2
5569,ham,"Pity, * was in mood for that. So...any other s...",0,57,10,4
5570,ham,The guy did some bitching but I acted like i'd...,0,125,26,1


In [ ]:
# prompt: plotly subplots to view the mean of num_chars, num_words, num_sentences where df['Category']=ham and where df['Category'] is spam.

import plotly.graph_objects as go

# Create subplots
fig = go.Figure()

# Calculate mean values for ham messages
ham_df = df[df['Category'] == 'ham']
ham_means = ham_df[['num_chars', 'num_words', 'num_sentences']].mean()

# Add bar chart for ham messages
fig.add_trace(go.Bar(
    x=ham_means.index,
    y=ham_means.values,
    name='Ham',
    marker_color='blue'
))

# Calculate mean values for spam messages
spam_df = df[df['Category'] == 'spam']
spam_means = spam_df[['num_chars', 'num_words', 'num_sentences']].mean()

# Add bar chart for spam messages
fig.add_trace(go.Bar(
    x=spam_means.index,
    y=spam_means.values,
    name='Spam',
    marker_color='red'
))

# Update layout
fig.update_layout(
    barmode='group',
    title='Mean of num_chars, num_words, num_sentences for Ham and Spam',
    xaxis_title='Features',
    yaxis_title='Mean Value'
)

# Show the plot
fig.show()


In [ ]:
# prompt: plotly subplots to view the mean of num_chars, num_words, num_sentences where df['Category']=ham and where df['Category'] is spam.

import plotly.graph_objects as go

# Create subplots
fig = go.Figure()

# Calculate mean values for ham messages
ham_df = df[df['Category'] == 'ham']
ham_means = ham_df[['num_chars', 'num_words', 'num_sentences']].median()

# Add bar chart for ham messages
fig.add_trace(go.Bar(
    x=ham_means.index,
    y=ham_means.values,
    name='Ham',
    marker_color='blue'
))

# Calculate mean values for spam messages
spam_df = df[df['Category'] == 'spam']
spam_means = spam_df[['num_chars', 'num_words', 'num_sentences']].median()

# Add bar chart for spam messages
fig.add_trace(go.Bar(
    x=spam_means.index,
    y=spam_means.values,
    name='Spam',
    marker_color='red'
))

# Update layout
fig.update_layout(
    barmode='group',
    title='Mean of num_chars, num_words, num_sentences for Ham and Spam',
    xaxis_title='Features',
    yaxis_title='Median Value'
)

# Show the plot
fig.show()


In [ ]:
# prompt: Using dataframe df: Using df, in the Message column, perform the following tasks:
# Removing punctuations
# Removing URLs
# Removing Stop words
# Lower casing
# And storing the preprocessed data into a new column "Cleaned_Message"

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already present
nltk.download('stopwords')

def preprocess_text(text):
    """
    Preprocesses text by removing punctuations, URLs, stop words, and converting to lower case.
    """
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    # Lower casing
    text = text.lower()

    return text

# Apply the preprocess_text function to the 'Message' column and store the result in 'Cleaned_Message'
df['Cleaned_Message'] = df['Message'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,Category,Message,Target,num_chars,num_words,num_sentences,Cleaned_Message
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,20,4,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,0,29,6,3,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,28,2,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,49,11,3,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,13,1,nah dont think goes usf lives around though
...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,160,30,6,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,Will ü b going to esplanade fr home?,0,36,8,2,ü b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",0,57,10,4,pity mood soany suggestions
5570,ham,The guy did some bitching but I acted like i'd...,0,125,26,1,guy bitching acted like id interested buying s...


In [ ]:
# # prompt: Using dataframe df: Tokenize Cleaned_Message

# # Import necessary library
# from nltk.tokenize import word_tokenize

# # Tokenize the 'Cleaned_Message' column
# df['Tokenized_Message'] = df['Cleaned_Message'].apply(lambda x: word_tokenize(x))

# df


In [ ]:
# prompt: Lemmatize df['Cleaned_Message']

# Download WordNetLemmatizer resources if not already present
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
  """Lemmatizes the text."""
  tokens = word_tokenize(text)
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  return " ".join(lemmatized_tokens)

df['Lemmatized_Message'] = df['Cleaned_Message'].apply(lemmatize_text)

df


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Category,Message,Target,num_chars,num_words,num_sentences,Cleaned_Message,Lemmatized_Message
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,20,4,go jurong point crazy available bugis n great ...,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,0,29,6,3,ok lar joking wif u oni,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,28,2,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,49,11,3,u dun say early hor u c already say,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,13,1,nah dont think goes usf lives around though,nah dont think go usf life around though
...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,160,30,6,2nd time tried 2 contact u u 750 pound prize 2...,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,Will ü b going to esplanade fr home?,0,36,8,2,ü b going esplanade fr home,ü b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",0,57,10,4,pity mood soany suggestions,pity mood soany suggestion
5570,ham,The guy did some bitching but I acted like i'd...,0,125,26,1,guy bitching acted like id interested buying s...,guy bitching acted like id interested buying s...


In [ ]:
df['Lemmatized_Message']

,Lemmatized_Message
0,go jurong point crazy available bugis n great ...
1,ok lar joking wif u oni
2,free entry 2 wkly comp win fa cup final tkts 2...
3,u dun say early hor u c already say
4,nah dont think go usf life around though
...,...
5567,2nd time tried 2 contact u u 750 pound prize 2...
5568,ü b going esplanade fr home
5569,pity mood soany suggestion
5570,guy bitching acted like id interested buying s...


In [ ]:
# prompt: Count plots using plotly for 15 most used words in ham and spam messages of df['Category']

from collections import Counter

# Separate ham and spam messages
ham_messages = df[df['Category'] == 'ham']['Lemmatized_Message']
spam_messages = df[df['Category'] == 'spam']['Lemmatized_Message']

# Combine all words from ham messages into a single list
ham_words = [word for message in ham_messages for word in message.split()]

# Combine all words from spam messages into a single list
spam_words = [word for message in spam_messages for word in message.split()]


# Count the most frequent words in ham messages
ham_word_counts = Counter(ham_words)
most_common_ham_words = ham_word_counts.most_common(15)


# Count the most frequent words in spam messages
spam_word_counts = Counter(spam_words)
most_common_spam_words = spam_word_counts.most_common(15)


import plotly.graph_objects as go

# Create a bar chart for ham messages
fig_ham = go.Figure(
    data=[go.Bar(
        x=[word for word, count in most_common_ham_words],
        y=[count for word, count in most_common_ham_words],
    )],
    layout=go.Layout(
        title='15 Most Frequent Words in Ham Messages',
        xaxis_title='Words',
        yaxis_title='Frequency'
    )
)

fig_ham.show()


# Create a bar chart for spam messages
fig_spam = go.Figure(
    data=[go.Bar(
        x=[word for word, count in most_common_spam_words],
        y=[count for word, count in most_common_spam_words],
    )],
    layout=go.Layout(
        title='15 Most Frequent Words in Spam Messages',
        xaxis_title='Words',
        yaxis_title='Frequency'
    )
)

fig_spam.show()


In [ ]:
# # prompt: Apply stemmer on df'Lemmatized_Message'

# from nltk.stem import PorterStemmer

# stemmer = PorterStemmer()

# def stem_tokens(tokens):
#   return [stemmer.stem(token) for token in tokens]

# df['Stemmed_Message'] = df['Tokenized_Message'].apply(stem_tokens)
# df


In [ ]:
y = df["Target"]
X=df["Lemmatized_Message"]

In [ ]:
tokenizer = Tokenizer() #initializing the tokenizer
tokenizer.fit_on_texts(X)# fitting on the sms data
text_to_sequence = tokenizer.texts_to_sequences(X) # creating the numerical sequence
text_to_sequence

[[7,
  3487,
  260,
  549,
  584,
  1011,
  44,
  62,
  322,
  1012,
  86,
  2325,
  1013,
  13,
  3488,
  71],
 [11, 208, 1151, 323, 1, 1501],
 [14,
  416,
  4,
  834,
  835,
  137,
  2326,
  1014,
  585,
  2327,
  2328,
  184,
  2329,
  22,
  2326,
  2330,
  271,
  416,
  3489,
  38,
  3490,
  340,
  3491],
 [1, 141, 58, 249, 2331, 1, 82, 73, 58],
 [773, 8, 37, 7, 709, 103, 120, 341],
 [649,
  54,
  1015,
  113,
  52,
  157,
  34,
  272,
  15,
  273,
  33,
  1502,
  11,
  250,
  1152,
  3492,
  27,
  311,
  2332],
 [134, 457, 15, 287, 491, 15, 2333, 3493],
 [158,
  906,
  2334,
  2334,
  2335,
  3494,
  3495,
  3496,
  492,
  1503,
  550,
  1153,
  417,
  1016,
  74,
  1503],
 [613,
  907,
  312,
  169,
  368,
  3497,
  908,
  89,
  1017,
  67,
  2,
  3498,
  67,
  324,
  3499,
  437,
  650,
  176],
 [47,
  909,
  194,
  1,
  35,
  1306,
  458,
  261,
  474,
  47,
  352,
  14,
  2,
  47,
  458,
  87,
  14,
  3500],
 [3, 127, 60, 30, 138, 8, 21, 177, 182, 910, 121, 106, 94, 3501, 325,

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_length_sequence = max([len(i) for i in text_to_sequence])
 # finding the length of largest sequence
padded_sms_sequence = pad_sequences(text_to_sequence, maxlen=max_length_sequence,
                                    padding = "pre") # padding the sequence
padded_sms_sequence

[[   0    0    0 ...   13 3488   71]
 [   0    0    0 ...  323    1 1501]
 [   0    0    0 ... 3490  340 3491]
 ...
 [   0    0    0 ... 1477 8855 3483]
 [   0    0    0 ...  785    1   14]
 [   0    0    0 ... 2077  428  186]]


In [ ]:
TOT_SIZE = len(tokenizer.word_index)+1
def create_model():

      lstm_model = Sequential()
      lstm_model.add(Embedding(TOT_SIZE, 32, input_length=max_length_sequence))
      lstm_model.add(LSTM(100))
      lstm_model.add(Dropout(0.4))
      lstm_model.add(Dense(20, activation="relu"))
      lstm_model.add(Dropout(0.3))
      lstm_model.add(Dense(1, activation = "sigmoid"))
      return lstm_model
lstm_model = create_model()
lstm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sms_sequence, y, test_size=0.2, random_state=42)

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.8502 - loss: 0.4332 - val_accuracy: 0.9651 - val_loss: 0.1344
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.9755 - loss: 0.0991 - val_accuracy: 0.9835 - val_loss: 0.0575
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9912 - loss: 0.0468 - val_accuracy: 0.9826 - val_loss: 0.0552
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.9969 - loss: 0.0163 - val_accuracy: 0.9806 - val_loss: 0.0720
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.9986 - loss: 0.0096 - val_accuracy: 0.9797 - val_loss: 0.0997
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.9989 - loss: 0.0084 - val_accuracy: 0.9806 - val_loss: 0.1001
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - accuracy: 0.9991 - loss: 0.0073 - val_accuracy: 0.9864 - val_loss: 0.0755
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - accuracy: 0.9988 - loss: 0.0054 - v

In [ ]:
lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 80, 32)              │         283,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          53,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │           2,020 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,015,997 (3.88 MB)

 Trainable params: 338,665 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 677,332 (2.58 MB)

In [ ]:
# prompt: Take Input of a phrase and predict using the model if the input is a spam or ham text.

def predict_spam_ham(input_phrase):
  """Predicts whether the input phrase is spam or ham."""

  # Preprocess the input phrase
  input_phrase = preprocess_text(input_phrase)
  input_phrase = lemmatize_text(input_phrase)

  # Tokenize and pad the input phrase
  input_sequence = tokenizer.texts_to_sequences([input_phrase])
  padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length_sequence, padding='pre')

  # Make the prediction
  prediction = lstm_model.predict(padded_input_sequence)[0][0]

  if prediction >= 0.5:
    print("Prediction: Spam")
  else:
    print("Prediction: Ham")

# Get user input
user_input = input("Enter a phrase: ")

# Predict and print the result
predict_spam_ham(user_input)


Enter a phrase: Win a free lottery by clicking this link
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Spam


In [ ]:
# prompt: Classify spam/ham messages using SVM and generate model accuracy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['Lemmatized_Message'], df['Target'], test_size=0.2, random_state=42
)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = vectorizer.transform(X_test)

# Create an SVM model
svm_model = SVC(kernel='linear')

# Train the model
svm_model.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = svm_model.predict(X_test_tfidf)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

print("SVM Model Accuracy:", accuracy)


SVM Model Accuracy: 0.9806201550387597


In [ ]:
# prompt: test the svm model by taking user input and predicting its category

def predict_svm(input_phrase):
  """Predicts whether the input phrase is spam or ham using the SVM model."""

  # Preprocess the input phrase
  input_phrase = preprocess_text(input_phrase)
  input_phrase = lemmatize_text(input_phrase)

  # Transform the input phrase using the TF-IDF vectorizer
  input_tfidf = vectorizer.transform([input_phrase])

  # Make the prediction
  prediction = svm_model.predict(input_tfidf)[0]

  if prediction == 1:
    print("Prediction: Spam")
  else:
    print("Prediction: Ham")


# Get user input
user_input = input("Enter a phrase: ")

# Predict and print the result
predict_svm(user_input)


Enter a phrase: Win a free lottery by clicking this link.
Prediction: Spam


In [ ]:
# prompt: Classify spam/ham messages using Naive Bayes and generate model accuracy

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['Lemmatized_Message'], df['Target'], test_size=0.2, random_state=42
)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = vectorizer.transform(X_test)

# Create a Naive Bayes model
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = nb_model.predict(X_test_tfidf)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

print("Naive Bayes Model Accuracy:", accuracy)

def predict_nb(input_phrase):
  """Predicts whether the input phrase is spam or ham using the Naive Bayes model."""

  # Preprocess the input phrase
  input_phrase = preprocess_text(input_phrase)
  input_phrase = lemmatize_text(input_phrase)

  # Transform the input phrase using the TF-IDF vectorizer
  input_tfidf = vectorizer.transform([input_phrase])

  # Make the prediction
  prediction = nb_model.predict(input_tfidf)[0]

  if prediction == 1:
    print("Prediction: Spam")
  else:
    print("Prediction: Ham")


# Get user input
user_input = input("Enter a phrase: ")

# Predict and print the result
predict_nb(user_input)


Naive Bayes Model Accuracy: 0.9602713178294574
Enter a phrase: Win a free lottery by clicking this link.
Prediction: Spam
